**This notebook has implementation of ELENA for solving the vehicle routing problem on Augerat-1995 dataset set A.**

---


**Structure:**

1. Imports
2. Data Loading
3. Parameter Initialization
4. Implementation
5. Visualizations


---

Imports

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import time
from typing import List, Tuple
from multiprocessing import Pool, cpu_count
from functools import lru_cache
import math
import random
import glob
import os
import re

Data Loading

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/DNA-Insipred NNs/Input/augerat-1995-set-a.zip'

Archive:  /content/drive/MyDrive/DNA-Insipred NNs/Input/augerat-1995-set-a.zip
  inflating: readme.txt              
  inflating: A-n32-k05.xml           
  inflating: A-n33-k05.xml           
  inflating: A-n33-k06.xml           
  inflating: A-n34-k05.xml           
  inflating: A-n36-k05.xml           
  inflating: A-n37-k05.xml           
  inflating: A-n37-k06.xml           
  inflating: A-n38-k05.xml           
  inflating: A-n39-k05.xml           
  inflating: A-n39-k06.xml           
  inflating: A-n44-k06.xml           
  inflating: A-n45-k06.xml           
  inflating: A-n45-k07.xml           
  inflating: A-n46-k07.xml           
  inflating: A-n48-k07.xml           
  inflating: A-n53-k07.xml           
  inflating: A-n54-k07.xml           
  inflating: A-n55-k09.xml           
  inflating: A-n60-k09.xml           
  inflating: A-n61-k09.xml           
  inflating: A-n62-k08.xml           
  inflating: A-n63-k09.xml           
  inflating: A-n63-k10.xml           
  inflati

Distance Calculation

In [ ]:
# Calculate total distance for VRP routes
@lru_cache(maxsize=None)
def calculate_distance(routes):
    total_distance = 0
    for route in routes:
        if route:  # Check if route is not empty
            # Add distance from depot to first city
            total_distance += dist_matrix[0, route[0]]
            # Add distance between cities
            total_distance += sum(dist_matrix[route[i], route[i+1]] for i in range(len(route)-1))
            # Add distance from last city back to depot
            total_distance += dist_matrix[route[-1], 0]
    return total_distance

ELENA Implementation: population size of 300 and initial mutation rate of 0.5

In [ ]:
class CompressedLP:
    def __init__(self, city_index):
        self.city_index = city_index
        self.epigenetic_tags = {
            'mutation_resistance': np.random.random(),
            'crossover_affinity': np.random.random(),
            'stability_score': np.random.random()
        }

class CompressedLPVRP:
    def __init__(self, num_cities, num_vehicles, capacity, demands, subpop_id=0):
        self.neurons = [CompressedLP(i) for i in range(1, num_cities)]  # Exclude depot
        self.num_vehicles = num_vehicles
        self.capacity = capacity
        self.demands = demands
        self.subpop_id = subpop_id
        self.fitness_history = []
        np.random.shuffle(self.neurons)
        self.needs_improvement = True

    def split_into_routes(self, sequence):
        routes = []
        current_route = []
        current_load = 0

        for city in sequence:
            if current_load + self.demands[city] <= self.capacity:
                current_route.append(city)
                current_load += self.demands[city]
            else:
                if current_route:
                    routes.append(tuple(current_route))
                current_route = [city]
                current_load = self.demands[city]

        if current_route:
            routes.append(tuple(current_route))

        # If we have more routes than vehicles, combine some routes
        while len(routes) > self.num_vehicles:
            # Find two shortest routes to combine
            routes.sort(key=len)
            combined = routes[0] + routes[1]
            routes = [combined] + routes[2:]

        return tuple(routes)

    def get_routes(self):
        sequence = [n.city_index for n in self.neurons]
        return self.split_into_routes(sequence)

    def two_opt(self, routes):
        improved = True
        best_distance = calculate_distance(routes)
        initial_distance = best_distance

        while improved:
            improved = False
            for route_idx, route in enumerate(routes):
                route = list(route)
                for i in range(len(route) - 1):
                    for j in range(i + 1, len(route)):
                        new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
                        new_routes = list(routes)
                        new_routes[route_idx] = tuple(new_route)
                        new_distance = calculate_distance(tuple(new_routes))

                        if new_distance < best_distance:
                            routes = tuple(new_routes)
                            best_distance = new_distance
                            improved = True

        improvement = initial_distance - best_distance
        self.update_epigenetic_tags(improvement)
        self.needs_improvement = False
        return routes, best_distance

    def update_epigenetic_tags(self, improvement):
        for neuron in self.neurons:
            if improvement > 0:
                neuron.epigenetic_tags['stability_score'] = min(1.0, neuron.epigenetic_tags['stability_score'] + 0.1)
                neuron.epigenetic_tags['mutation_resistance'] = min(1.0, neuron.epigenetic_tags['mutation_resistance'] + 0.05)
            else:
                neuron.epigenetic_tags['stability_score'] = max(0.0, neuron.epigenetic_tags['stability_score'] - 0.1)
                neuron.epigenetic_tags['mutation_resistance'] = max(0.0, neuron.epigenetic_tags['mutation_resistance'] - 0.05)

    def fitness(self):
        routes = self.get_routes()
        distance = calculate_distance(routes)
        return 1 / distance, routes

    def mutate(self):
        self.needs_improvement = True
        mutation_type = np.random.choice(['swap', 'insert', 'reverse'])

        eligible_indices = [i for i, n in enumerate(self.neurons)
                          if n.epigenetic_tags['mutation_resistance'] < np.random.random()]

        if len(eligible_indices) < 2:
            return

        if mutation_type == 'swap':
            i, j = np.random.choice(eligible_indices, 2, replace=False)
            self.neurons[i], self.neurons[j] = self.neurons[j], self.neurons[i]
        elif mutation_type == 'insert':
            i, j = np.random.choice(eligible_indices, 2, replace=False)
            city = self.neurons.pop(i)
            self.neurons.insert(j, city)
        else:  # reverse
            i, j = sorted(np.random.choice(eligible_indices, 2, replace=False))
            self.neurons[i:j+1] = reversed(self.neurons[i:j+1])

def crossover(parent1, parent2):
    child = CompressedLPVRP(
        len(parent1.neurons) + 1,  # Add 1 for depot
        parent1.num_vehicles,
        parent1.capacity,
        parent1.demands,
    )

    crossover_points = [i for i, n in enumerate(parent1.neurons)
                       if n.epigenetic_tags['crossover_affinity'] > 0.5]

    if len(crossover_points) >= 2:
        start, end = sorted(np.random.choice(crossover_points, 2, replace=False))
    else:
        start, end = sorted(np.random.choice(len(parent1.neurons), 2, replace=False))

    segment = parent1.neurons[start:end]
    child_cities = set(n.city_index for n in segment)
    remaining = [n for n in parent2.neurons if n.city_index not in child_cities]

    child.neurons = remaining[:start] + segment + remaining[start:]
    child.needs_improvement = True
    return child

def horizontal_gene_transfer(subpopulations):
    for i, subpop1 in enumerate(subpopulations):
        for j, subpop2 in enumerate(subpopulations):
            if i != j:
                for ind1 in subpop1:
                    for ind2 in subpop2:
                        if np.random.random() < 0.1:
                            stable_segment = find_stable_segment(ind1)
                            if stable_segment:
                                transfer_segment(ind2, stable_segment)

def find_stable_segment(individual):
    stability_threshold = 0.7
    stable_positions = [i for i, n in enumerate(individual.neurons)
                       if n.epigenetic_tags['stability_score'] > stability_threshold]

    if len(stable_positions) < 2:
        return None

    start = stable_positions[0]
    length = 1
    for i in range(1, len(stable_positions)):
        if stable_positions[i] == stable_positions[i-1] + 1:
            length += 1
        else:
            break

    if length >= 2:
        return individual.neurons[start:start+length]
    return None

def transfer_segment(recipient, segment):
    segment_cities = set(n.city_index for n in segment)
    recipient.neurons = [n for n in recipient.neurons if n.city_index not in segment_cities]
    insert_pos = np.random.randint(0, len(recipient.neurons))
    recipient.neurons[insert_pos:insert_pos] = segment

def process_individual(network):
    if np.random.random() < 0.5:
        network.mutate()

    if network.needs_improvement:
        routes = network.get_routes()
        routes, _ = network.two_opt(routes)
        # Reconstruct neurons from routes
        sequence = [city for route in routes for city in route]
        network.neurons = [CompressedLP(city) for city in sequence]

    return network.fitness()[0], network

def evolve_population(pop_size, generations, early_stopping_generations, num_cities, num_vehicles, capacity, demands, num_subpops=3):
    subpopulations = [
        [CompressedLPVRP(num_cities, num_vehicles, capacity, demands, subpop_id=i)
         for _ in range(pop_size // num_subpops)]
        for i in range(num_subpops)
    ]

    best_fitness = 0
    best_fitness_count = 0

    with Pool(processes=cpu_count()) as pool:
        for gen in range(generations):
            for subpop in subpopulations:
                fitnesses_and_networks = pool.map(process_individual, subpop)
                sorted_population = sorted(fitnesses_and_networks, key=lambda x: x[0], reverse=True)
                subpop[:] = [network for _, network in sorted_population]

            if gen % 5 == 0:
                horizontal_gene_transfer(subpopulations)

            current_best_fitness = max(
                subpop[0].fitness()[0] for subpop in subpopulations
            )
            avg_fitness = np.mean([ind.fitness()[0] for subpop in subpopulations for ind in subpop])

            print(f"Generation {gen}: Best Fitness = {current_best_fitness:.4f}, Avg Fitness = {avg_fitness:.4f}")

            if current_best_fitness > best_fitness:
                best_fitness = current_best_fitness
                best_fitness_count = 0
            else:
                best_fitness_count += 1

            if best_fitness_count >= early_stopping_generations:
                print(f"Early stopping at generation {gen}")
                break

            for subpop in subpopulations:
                elite_size = len(subpop) // 10
                new_population = list(subpop[:elite_size])

                while len(new_population) < len(subpop):
                    if np.random.random() < 0.7:
                        tournament_size = 5
                        parent1 = max(np.random.choice(subpop, tournament_size),
                                    key=lambda x: x.fitness()[0])
                        parent2 = max(np.random.choice(subpop, tournament_size),
                                    key=lambda x: x.fitness()[0])
                        child = crossover(parent1, parent2)
                    else:
                        child = np.random.choice(subpop[:len(subpop)//2])

                    new_population.append(child)

                subpop[:] = new_population

    return max(
        (max(subpop, key=lambda x: x.fitness()[0]) for subpop in subpopulations),
        key=lambda x: x.fitness()[0]
    )

def run_evolutionary_algorithm(num_cities, num_vehicles, capacity, pop_size=50, generations=50, early_stopping_generations=5):
    global cities, dist_matrix, demands

    # Generate random cities (depot is at index 0)
    cities = np.random.rand(num_cities, 2)
    dist_matrix = squareform(pdist(cities))

    # Generate random demands for each city (excluding depot)
    demands = np.zeros(num_cities)
    demands[1:] = np.random.randint(1, 11, size=num_cities-1)

    print(f"\nRunning VRP Evolutionary Algorithm for {num_cities} cities and {num_vehicles} vehicles...")
    start_time = time.time()

    # Initialize and evolve population
    subpopulations = [
        [CompressedLPVRP(num_cities, num_vehicles, capacity, demands, subpop_id=i)
         for _ in range(pop_size // 3)]
        for i in range(3)
    ]

    best_fitness = 0
    best_fitness_count = 0

    with Pool(processes=cpu_count()) as pool:
        for gen in range(generations):
            for subpop in subpopulations:
                fitnesses_and_networks = pool.map(process_individual, subpop)
                sorted_population = sorted(fitnesses_and_networks, key=lambda x: x[0], reverse=True)
                subpop[:] = [network for _, network in sorted_population]

            if gen % 5 == 0:
                horizontal_gene_transfer(subpopulations)

            current_best_fitness = max(
                subpop[0].fitness()[0] for subpop in subpopulations
            )
            avg_fitness = np.mean([ind.fitness()[0] for subpop in subpopulations for ind in subpop])

            print(f"Generation {gen}: Best Fitness = {current_best_fitness:.4f}, Avg Fitness = {avg_fitness:.4f}")

            if current_best_fitness > best_fitness:
                best_fitness = current_best_fitness
                best_fitness_count = 0
            else:
                best_fitness_count += 1

            if best_fitness_count >= early_stopping_generations:
                print(f"Early stopping at generation {gen}")
                break

            for subpop in subpopulations:
                elite_size = len(subpop) // 10
                new_population = list(subpop[:elite_size])

                while len(new_population) < len(subpop):
                    if np.random.random() < 0.7:
                        tournament_size = 5
                        parent1 = max(np.random.choice(subpop, tournament_size),
                                    key=lambda x: x.fitness()[0])
                        parent2 = max(np.random.choice(subpop, tournament_size),
                                    key=lambda x: x.fitness()[0])
                        child = crossover(parent1, parent2)
                    else:
                        child = np.random.choice(subpop[:len(subpop)//2])

                    new_population.append(child)

                subpop[:] = new_population

    # Get the best solution from all subpopulations
    best_solution = max(
        (max(subpop, key=lambda x: x.fitness()[0]) for subpop in subpopulations),
        key=lambda x: x.fitness()[0]
    )

    fitness, routes = best_solution.fitness()
    distance = 1 / fitness

    end_time = time.time()
    execution_time = end_time - start_time

    # Plot results
    plt.figure(figsize=(10, 6))
    plt.scatter(cities[0, 0], cities[0, 1], c='green', s=100, label='Depot')
    plt.scatter(cities[1:, 0], cities[1:, 1], c='red', s=50, label='Cities')

    colors = plt.cm.rainbow(np.linspace(0, 1, len(routes)))
    for route, color in zip(routes, colors):
        if len(route) > 0:  # Only plot if route is not empty
            # Plot from depot to first city
            plt.plot([cities[0, 0], cities[route[0], 0]],
                    [cities[0, 1], cities[route[0], 1]], c=color)

            # Plot between cities
            for i in range(len(route)-1):
                start = cities[route[i]]
                end = cities[route[i+1]]
                plt.plot([start[0], end[0]], [start[1], end[1]], c=color)

            # Plot from last city back to depot
            plt.plot([cities[route[-1], 0], cities[0, 0]],
                    [cities[route[-1], 1], cities[0, 1]], c=color)

    plt.title(f"VRP Solution - {num_cities} cities, {num_vehicles} vehicles\n" +
             f"Total Distance: {distance:.4f}, Time: {execution_time:.2f}s")
    plt.xlabel("X coordinate")
    plt.ylabel("Y coordinate")
    plt.legend()
    plt.show()

    return {
        "distance": distance,
        "time": execution_time,
        "routes": routes
    }

In [ ]:
def parse_xml_file(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Parse nodes
    nodes = []
    for node in root.findall('.//node'):
        node_id = int(node.get('id'))
        node_type = int(node.get('type'))
        cx = float(node.find('cx').text)
        cy = float(node.find('cy').text)
        nodes.append((node_id, node_type, cx, cy))

    # Parse vehicle capacity
    capacity = float(root.find('.//vehicle_profile/capacity').text)

    # Extract number of vehicles from filename (e.g., k05 means 5 vehicles)
    filename = os.path.basename(file_path)
    match = re.search(r'-k(\d+)', filename)
    if match:
        num_vehicles = int(match.group(1))  # Convert k05 to 5
    else:
        # Fallback to counting vehicle profiles if filename pattern doesn't match
        num_vehicles = len(root.findall('.//vehicle_profile'))

    # Parse demands
    demands = [0]  # Depot has no demand
    for request in root.findall('.//request'):
        demands.append(float(request.find('quantity').text))

    return nodes, capacity, num_vehicles, demands

def create_distance_matrix(nodes):
    coordinates = np.array([(x, y) for _, _, x, y in nodes])
    return squareform(pdist(coordinates))

def solve_vrp(file_path):
    # Parse XML data
    nodes, capacity, num_vehicles, demands = parse_xml_file(file_path)
    num_cities = len(nodes)

    # Create distance matrix
    global dist_matrix
    dist_matrix = create_distance_matrix(nodes)

    # Algorithm parameters
    pop_size = 300
    generations = 25
    early_stopping_generations = 5

    print(f"\nSolving {os.path.basename(file_path)}")
    print(f"Number of cities: {num_cities}")
    print(f"Number of vehicles: {num_vehicles}")
    print(f"Vehicle capacity: {capacity}")

    # Start timer
    start_time = time.time()

    # Run evolution and get best solution
    best_individual = evolve_population(
        pop_size=pop_size,
        generations=generations,
        early_stopping_generations=early_stopping_generations,
        num_cities=num_cities,
        num_vehicles=num_vehicles,
        capacity=capacity,
        demands=demands
    )

    # Get the best routes
    best_routes = best_individual.get_routes()

    # Plot the solution
    plot_solution(nodes, best_routes)

    # Print route details
    print("\nBest solution routes:")
    total_distance = 0
    for i, route in enumerate(best_routes, 1):
        route_distance = calculate_route_distance(route)
        total_distance += route_distance
        print(f"Route {i}: {[0] + list(route) + [0]} (Distance: {route_distance:.2f})")
    print(f"Total distance: {total_distance:.2f}")

    # Print execution time
    execution_time = time.time() - start_time
    print(f"Execution time: {execution_time:.2f} seconds")

def calculate_route_distance(route):
    distance = dist_matrix[0, route[0]]  # Depot to first city
    for i in range(len(route)-1):
        distance += dist_matrix[route[i], route[i+1]]
    distance += dist_matrix[route[-1], 0]  # Last city to depot
    return distance

def plot_solution(nodes, routes):
    plt.figure(figsize=(12, 8))

    # Plot nodes
    coordinates = np.array([(x, y) for _, _, x, y in nodes])
    plt.scatter(coordinates[1:, 0], coordinates[1:, 1], c='blue', s=100, label='Customers')
    plt.scatter(coordinates[0, 0], coordinates[0, 1], c='red', marker='s', s=200, label='Depot')

    # Plot routes with different colors
    colors = plt.cm.rainbow(np.linspace(0, 1, len(routes)))
    for i, (route, color) in enumerate(zip(routes, colors)):
        route_coords = coordinates[[0] + list(route) + [0]]
        plt.plot(route_coords[:, 0], route_coords[:, 1], c=color, linewidth=2, label=f'Route {i+1}')

        # Add arrows to show direction
        for j in range(len(route_coords)-1):
            mid_x = (route_coords[j][0] + route_coords[j+1][0]) / 2
            mid_y = (route_coords[j][1] + route_coords[j+1][1]) / 2
            plt.arrow(mid_x, mid_y,
                     (route_coords[j+1][0] - route_coords[j][0])/10,
                     (route_coords[j+1][1] - route_coords[j][1])/10,
                     head_width=1, head_length=1, fc=color, ec=color)

    # Add node labels
    for i, coord in enumerate(coordinates):
        plt.annotate(f'N{i}', (coord[0], coord[1]), xytext=(5, 5),
                    textcoords='offset points')

    plt.title('VRP Solution')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def main():
    # Get all XML files
    xml_files = glob.glob('A-n*.xml')

    for file_path in sorted(xml_files):
        solve_vrp(file_path)

if __name__ == "__main__":
    main()

Output hidden; open in https://colab.research.google.com to view.